<h1> Scaling up ML using Cloud AI Platform</h1>

In this notebook, we take a previously developed TensorFlow model to predict taxifare rides and package it up so that it can be run in Cloud AI Platform. For now, we'll run this on a small dataset. The model that was developed is rather simplistic, and therefore, the accuracy of the model is not great either.  However, this notebook illustrates *how* to package up a TensorFlow model to run it within Cloud AI Platform. 

Later in the course, we will look at ways to make a more effective machine learning model.

<h2> Environment variables for project and bucket </h2>

Note that:
<ol>
<li> Your project id is the *unique* string that identifies your project (not the project name). You can find this from the GCP Console dashboard's Home page.  My dashboard reads:  <b>Project ID:</b> cloud-training-demos </li>
<li> Cloud training often involves saving and restoring model files. If you don't have a bucket already, I suggest that you create one from the GCP console (because it will dynamically check whether the bucket name you want is available). A common pattern is to prefix the bucket name by the project id, so that it is unique. Also, for cost reasons, you might want to use a single region bucket. </li>
</ol>
<b>Change the cell below</b> to reflect your Project ID and bucket name.


In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [1]:
import tensorflow as tf
tf_version = tf.__version__

In [2]:
import os
PROJECT = 'cloud-training-281409' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'cloud-training-281409' # REPLACE WITH YOUR BUCKET NAME
REGION = 'europe-west2' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

In [83]:
# For Python Code
# Model Info
MODEL_NAME = 'taxifare'
# Model Version
MODEL_VERSION = 'v1'
# Training Directory name
TRAINING_DIR = 'taxi_trained'

In [99]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TRAINING_DIR'] = TRAINING_DIR 
#os.environ['TFVERSION'] = '2.2.0'  # Tensorflow version
os.environ['TFVERSION'] = tf_version  # Tensorflow version

In [4]:
#%%bash
!gcloud config set project $PROJECT
!gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [27]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=2LABhT7T8R-2TV1z8jDbnOgwHYo7NKJqFUDhv_XsiRo&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account



You are now logged in as [enkidupal@gmail.com].
Your current project is [cloud-training-281409].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [11]:
!gcloud config set account enkidupal@gmail.com

Updated property [core/account].


In [74]:
AUTH_TOKEN_RESPONSE= !gcloud auth print-access-token
AUTH_TOKEN = AUTH_TOKEN_RESPONSE[0]

print(AUTH_TOKEN)

PROJECT_ID= PROJECT

command = f'curl -X GET -s -H "Content-Type: application/json" \
            -H "Authorization: Bearer {AUTH_TOKEN}" \
            https://ml.googleapis.com/v1/projects/{PROJECT_ID}:getConfig \
            | python -c "import json; import sys; response = json.load(sys.stdin); \
            print(response[\'serviceAccount\'])"'

SVC_ACCOUNT_OUTPUT= !$command
SVC_ACCOUNT = SVC_ACCOUNT_OUTPUT[0]

!echo "Authorizing the Cloud AI Platform account {SVC_ACCOUNT} to access files in {BUCKET}"
!gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
!gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
!gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET


ya29.a0AfH6SMAW4XL5PcJR2cxu0iCU5BjOYpk5EeoPbln6Vk4-EMBqfvN_PPS047yMXb3rgTKhjvwcL7Ue06S-g2wdIq3FmrKLTdSguXuqKLb2hbc5Y0TC12_x2MimSISSf99dzqxvXUinXMyF_MsDR44eH1-PsDeXgnGRWkO0KceOhGI
"Authorizing the Cloud AI Platform account service-389426224026@cloud-ml.google.com.iam.gserviceaccount.com to access files in cloud-training-281409"


Updated default ACL on gs://cloud-training-281409/
CommandException: "acl" command does not support "file://" URLs. Did you mean to use a gs:// URL?
Updated ACL on gs://cloud-training-281409/


In [ ]:
cmd = f'gsutil -m acl ch -u {SVC_ACCOUNT}:R -r gs://{BUCKET}'

Allow the Cloud AI Platform service account to read/write to the bucket containing training data.

In [ ]:
%%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print(response['serviceAccount'])")

echo "Authorizing the Cloud AI Platform account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

<h2> Packaging up the code </h2>

Take your code and put into a standard Python package structure.  <a href="taxifare/trainer/model.py">model.py</a> and <a href="taxifare/trainer/task.py">task.py</a> containing the Tensorflow code from earlier (explore the <a href="taxifare/trainer/">directory structure</a>).

In [5]:
%%bash
## check whether there are anymore TODOs 
## exit with 0 to avoid notebook process error
grep TODO taxifare/trainer/*.py; rc=$?

case $rc in 
    0) ;;
    1) echo "No more TODOs!"; exit 0;;
esac

No more TODOs!


In [100]:
!echo {MODEL_NAME}

taxifare


In [ ]:
%%bash
cat ${MODEL_NAME}/trainer/model.py

<h2> Find absolute paths to your data </h2>

Note the absolute paths below. /content is mapped in Datalab to where the home icon takes you

In [109]:
pwd = !pwd
pwd = pwd[0]
csv_folder = f'{pwd}/..'
!echo $pwd
!rm -rf $pwd/taxi_trained

!head -1 $csv_folder/taxi-train.csv
!head -1 $csv_folder/taxi-valid.csv

/c/Users/michal/PycharmProjects/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/labs
12.0,-73.987625,40.750617,-73.971163,40.78518,1,0
6.0,-74.013667,40.713935,-74.007627,40.702992,2,0


<h2> Running the Python module from the command-line </h2>

In [121]:
#%%bash
!rm -rf taxifare.tar.gz taxi_trained
#!rm -rf $pwd/$TRAINING_DIR
#!export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare
cmd = f'''python -m {TRAINING_DIR}.trainer.task \
   --train_data_paths="{csv_folder}/taxi-train*" \
   --eval_data_paths={csv_folder}/taxi-valid.csv  \
   --output_dir={pwd}/{TRAINING_DIR} \
   --train_steps=100 --job-dir=./tmp
'''

print(f'Executing command: \n{cmd}')
      
!cmd

Executing command: 
python -m taxi_trained.trainer.task    --train_data_paths="/c/Users/michal/PycharmProjects/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/labs/../taxi-train*"    --eval_data_paths=/c/Users/michal/PycharmProjects/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/labs/../taxi-valid.csv     --output_dir=/c/Users/michal/PycharmProjects/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/labs/taxi_trained    --train_steps=100 --job-dir=./tmp

^C


In [122]:
!python -m taxi_trained.trainer.task

c:\Users\michal\Anaconda3\envs\ML-37\python.exe: Error while finding module specification for 'taxi_trained.trainer.task' (ModuleNotFoundError: No module named 'taxi_trained')


In [ ]:
%%bash
ls $PWD/taxi_trained/export/exporter/

In [ ]:
%%writefile ./test.json
{"pickuplon": -73.885262,"pickuplat": 40.773008,"dropofflon": -73.987232,"dropofflat": 40.732403,"passengers": 2}

In [ ]:
%%bash
sudo find "/usr/lib/google-cloud-sdk/lib/googlecloudsdk/command_lib/ml_engine" -name '*.pyc' -delete

In [ ]:
%%bash
model_dir=$(ls ${PWD}/taxi_trained/export/exporter)
gcloud ai-platform local predict \
    --model-dir=${PWD}/taxi_trained/export/exporter/${model_dir} \
    --json-instances=./test.json

<h2> Running locally using gcloud </h2>

In [ ]:
%%bash
rm -rf taxifare.tar.gz taxi_trained
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   -- \
   --train_data_paths=${PWD}/taxi-train.csv \
   --eval_data_paths=${PWD}/taxi-valid.csv  \
   --train_steps=1000 \
   --output_dir=${PWD}/taxi_trained 

When I ran it (due to random seeds, your results will be different), the ```average_loss``` (Mean Squared Error) on the evaluation dataset was 187, meaning that the RMSE was around 13.

In [ ]:
!ls $PWD/taxi_trained

In [ ]:
%%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/taxifare/smallinput/
gsutil -m cp ${PWD}/*.csv gs://${BUCKET}/taxifare/smallinput/

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/taxifare/smallinput/taxi_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version 2.1 \
   --python-version 3.5 \
   -- \
   --train_data_paths="gs://${BUCKET}/taxifare/smallinput/taxi-train*" \
   --eval_data_paths="gs://${BUCKET}/taxifare/smallinput/taxi-valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

Don't be concerned if the notebook appears stalled (with a blue progress bar) or returns with an error about being unable to refresh auth tokens. This is a long-lived Cloud job and work is going on in the cloud. 

<b>Use the Cloud Console link to monitor the job and do NOT proceed until the job is done.</b>

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/taxifare/smallinput/

<h2> Train on larger dataset </h2>

I have already followed the steps below and the files are already available. <b> You don't need to do the steps in this comment. </b> In the next chapter (on feature engineering), we will avoid all this manual processing by using Cloud Dataflow.

Go to http://bigquery.cloud.google.com/ and type the query:
<pre>
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  'nokeyindata' AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  AND ABS(HASH(pickup_datetime)) % 1000 == 1
</pre>

Note that this is now 1,000,000 rows (i.e. 100x the original dataset).  Export this to CSV using the following steps (Note that <b>I have already done this and made the resulting GCS data publicly available</b>, so you don't need to do it.):
<ol>
<li> Click on the "Save As Table" button and note down the name of the dataset and table.
<li> On the BigQuery console, find the newly exported table in the left-hand-side menu, and click on the name.
<li> Click on "Export Table"
<li> Supply your bucket name and give it the name train.csv (for example: gs://cloud-training-demos-ml/taxifare/ch3/train.csv). Note down what this is.  Wait for the job to finish (look at the "Job History" on the left-hand-side menu)
<li> In the query above, change the final "== 1" to "== 2" and export this to Cloud Storage as valid.csv (e.g.  gs://cloud-training-demos-ml/taxifare/ch3/valid.csv)
<li> Download the two files, remove the header line and upload it back to GCS.
</ol>

<p/>
<p/>

<h2> Run Cloud training on 1-million row dataset </h2>

This took 60 minutes and uses as input 1-million rows.  The model is exactly the same as above. The only changes are to the input (to use the larger dataset) and to the Cloud MLE tier (to use STANDARD_1 instead of BASIC -- STANDARD_1 is approximately 10x more powerful than BASIC).  At the end of the training the loss was 32, but the RMSE (calculated on the validation dataset) was stubbornly at 9.03. So, simply adding more data doesn't help.

In [ ]:
%%bash

OUTDIR=gs://${BUCKET}/taxifare/ch3/taxi_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
CRS_BUCKET=cloud-training-demos # use the already exported data
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   --runtime-version 2.1 \
   --python-version 3.5 \
   -- \
   --train_data_paths="gs://${CRS_BUCKET}/taxifare/ch3/train.csv" \
   --eval_data_paths="gs://${CRS_BUCKET}/taxifare/ch3/valid.csv"  \
   --output_dir=$OUTDIR \
   --train_steps=100000

## Challenge Exercise

Modify your solution to the challenge exercise in d_trainandevaluate.ipynb appropriately. Make sure that you implement training and deployment. Increase the size of your dataset by 10x since you are running on the cloud. Does your accuracy improve?

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License